In [1]:
import os
import hardposit as hp
import fixed2float as fi
from pathlib import Path
import re

In [2]:
!date

Mon  8 May 2023 07:26:01 CEST


In [3]:
import fixed2float
fixed2float.__version__

'4.0.0-rc1'

In [4]:
N, ES = 16, 1
M, B = 31, 64

In [5]:
with open(Path("../tb_core_op_fma.log"), 'r') as f:
    log = f.read()
print(log)

0x6a00
(0x3524, 0x5e81) 0x0000000c84df2400
(0xd609, 0x5663) 0x0000000b623ce180
(0x7b0d, 0x998d) 0xfffffa88c75ce180
(0x8465, 0x5212) 0xfffff87f902ce180



In [6]:
logs = log.splitlines()
print(logs)

['0x6a00', '(0x3524, 0x5e81) 0x0000000c84df2400', '(0xd609, 0x5663) 0x0000000b623ce180', '(0x7b0d, 0x998d) 0xfffffa88c75ce180', '(0x8465, 0x5212) 0xfffff87f902ce180']


In [9]:
pattern = r"0x[0-9a-z]+"

acc = 0
for item in logs:
    results = re.findall(pattern, item)
    if len(results) == 1:
        p3 = hp.from_bits(int(results[0], 16), N, ES)
        acc += p3.eval()
    else:
        p1 = hp.from_bits(int(results[0], 16), N, ES)
        p2 = hp.from_bits(int(results[1], 16), N, ES)
        print(p1.eval(), p2.eval(), (p1*p2).eval())
        
        fixed = fi.from_bits(int(results[2], 16), M, B)
        print(f"{fixed=}")
        acc += p1.eval() * p2.eval()
        print(f"{acc=}")

        assert acc == fixed.eval()

0.66064453125 3.81298828125 2.51904296875
fixed=0000000000000000000000000000110010000100110111110010010000000000 Fx<31,64>(53768823808) 12.51902985572815
acc=12.51902985572815
-0.40570068359375 2.79833984375 -1.13525390625
fixed=0000000000000000000000000000101101100010001111001110000110000000 Fx<31,64>(48892797312) 11.383741468191147
acc=11.383741468191147
195.25 -7.224609375 -1408.0
fixed=1111111111111111111110101000100011000111010111001110000110000000 Fx<31,64>(18446738064100090240) -1399.2212390005589
acc=-1399.2212390005589
-230.75 2.2587890625 -522.0
fixed=1111111111111111111110000111111110010000001011001110000110000000 Fx<31,64>(18446735825496236416) -1920.4368151724339
acc=-1920.4368151724339
